# 1 Baby Steps - Algebra
------------------------

In [ ]:
import theano
import numpy as np
from theano import tensor
import seaborn as sns
sns.set_style('whitegrid')

In [ ]:
%matplotlib inline

## 1.1 Adding two Scalars

In [ ]:
a = tensor.dscalar()
b = tensor.dscalar()
c = a + b

In [ ]:
f = theano.function([a, b], c)

In [ ]:
%timeit f(1.5, 2.5)

## 1.2 Adding two Matrices

In [ ]:
x = tensor.dmatrix('x')
y = tensor.dmatrix('y')
z = x + y
f = theano.function([x, y], z)

In [ ]:
%timeit f([[1, 2], [3, 4]], [[10, 20], [30, 40]])

## 1.3 Exercise

In [ ]:
a = tensor.vector()
b = tensor.vector()
out = a ** 2 + b ** 2 + 2 * a * b
f = theano.function([a, b], out)

In [ ]:
%timeit f([0, 1, 2], [1, 2, 3])

# 2 More Examples
-----------------

## 2.1 Logistic Function

In [ ]:
x = tensor.dmatrix('x')
s = 1. / (1. + tensor.exp(-x))
logistic = theano.function([x], s)

In [ ]:
%timeit logistic([[0, 1], [-1, -2]])

In [ ]:
s2 = (1. + tensor.tanh(x/2)) / 2.0
logistic2 = theano.function([x], s2)

In [ ]:
%timeit logistic2([[0, 1], [-1, -2]])

In [ ]:
x = tensor.vector()
s3 = 1. / (1. + tensor.exp(-x))
logistic3 = theano.function([x], s3)

In [ ]:
x_data = np.linspace(-6, 6, 100, dtype='float32')
%timeit logistic3(x_data)
res = logistic3(x_data)

In [ ]:
sns.plt.plot(x_data, res)

## 2.2 Computing More than one Thing at the Same Time

In [ ]:
a, b = tensor.dmatrices('a', 'b')
diff = a - b
abs_diff = abs(diff)
diff_squared = abs_diff ** 2
f = theano.function([a, b], [diff, abs_diff, diff_squared])

In [ ]:
%timeit f([[1, 1], [1, 1]], [[0, 1], [2, 3]])

## 2.3 Setting a Default Value for an Argument

In [ ]:
from theano import In

In [ ]:
x, y = tensor.dscalars('x', 'y')
z = x + y
f = theano.function([x, In(y, value=1)], z)

In [ ]:
%timeit f(33)

In [ ]:
x, y, w = tensor.dscalars('x', 'y', 'w')
z = (x + y) * w
f = theano.function([x, In(y, value=1), In(w, value=2, name='w_by_name')], z)

In [ ]:
f(33)

In [ ]:
f(33, 2)

In [ ]:
f(33, 0, 1)

In [ ]:
f(33, w_by_name=2)

In [ ]:
f(33, w_by_name=2, y=1)

In [ ]:
f(x=33, w_by_name=2, y=1)

## 2.4 Using Shared Variables

In [ ]:
from theano import shared

In [ ]:
state = shared(0)
inc = tensor.iscalar('inc')
accumulator = theano.function([inc], state, updates=[(state, state+inc)])

In [ ]:
accumulator(1)

In [ ]:
state.get_value()

In [ ]:
accumulator(2)

In [ ]:
state.get_value()

In [ ]:
state.set_value(-2)

In [ ]:
accumulator(3)
state.get_value()

In [ ]:
fn_of_state = state * 2 + inc
foo = tensor.scalar(dtype=state.dtype)
skip_shared = theano.function([inc, foo], fn_of_state, givens=[(state, foo)])

In [ ]:
skip_shared(1, 3)

In [ ]:
state.get_value()

## 2.5 Coping functions

In [ ]:
state = shared(0)
inc = tensor.iscalar('inc')
accumulator = theano.function([inc], state, updates=[(state, state+inc)])

In [ ]:
new_state = shared(0)
new_accumulator = accumulator.copy(swap={state: new_state})

In [ ]:
new_accumulator(3)

In [ ]:
new_state.get_value()

In [ ]:
state.get_value()

In [ ]:
null_accumulator = accumulator.copy(delete_updates=True)

In [ ]:
null_accumulator(100)

In [ ]:
null_accumulator(1000)

In [ ]:
print(state.get_value())

## 2.6 Using Random Numbers

### *2.6.1 Brief Example*

In [ ]:
from theano.tensor.shared_randomstreams import RandomStreams

In [ ]:
srng = RandomStreams(seed=234)
rv_u = srng.uniform((2, 2))
rv_n = srng.normal((2, 2))
f = theano.function([], rv_u)
g = theano.function([], rv_n, no_default_updates=True)
nearly_zeros = theano.function([], rv_u + rv_u - 2 * rv_u)

In [ ]:
f_val0 = f()
f_val1 = f()

In [ ]:
print(f_val0)
print(f_val1)

In [ ]:
g_val0 = g()
g_val1 = g()

In [ ]:
print(g_val0)
print(g_val1)

### 2.6.2 *Seeding Streams*

In [ ]:
rng_val = rv_u.rng.get_value(borrow=True)
rng_val.seed(89234) 
rv_u.rng.set_value(rng_val, borrow=True)

In [ ]:
srng.seed(902340) 

### *2.6.3 Sharing Streams Between Functions*

In [ ]:
state_after_v0 = rv_u.rng.get_value().get_state()

In [ ]:
nearly_zeros()

In [ ]:
v1 = f()
rng = rv_u.rng.get_value(borrow=True)
rng.set_state(state_after_v0)
rv_u.rng.set_value(rng, borrow=True)

In [ ]:
v2 = f()
v3 = f()

In [ ]:
v2

In [ ]:
v3

In [ ]:
v1

### 2.6.4 *Copying Random State Between Theano Graphs*

In [ ]:
from theano.sandbox.rng_mrg import MRG_RandomStreams

In [ ]:
class Graph():
     def __init__(self, seed=123):
            self.rng = RandomStreams(seed)
            self.y = self.rng.uniform(size=(1,))

In [ ]:
g1 = Graph(seed=123)
f1 = theano.function([], g1.y)

In [ ]:
g2 = Graph(seed=987)
f2 = theano.function([], g2.y)

In [ ]:
f1()

In [ ]:
f2()

In [ ]:
def copy_random_state(g1, g2):
    if isinstance(g1.rng, MRG_RandomStreams):
        g2.rng.rstate = g1.rng.rstate
    for (su1, su2) in zip(g1.rng.state_updates, g2.rng.state_updates):
        su2[0].set_value(su1[0].get_value())

In [ ]:
copy_random_state(g1, g2)

In [ ]:
f1()

In [ ]:
f2()